<a href="https://colab.research.google.com/github/BoKna031/Image_inpainting/blob/master/Image_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect to Kaggle dataset

In [ ]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list

In [6]:
!kaggle datasets download -d amanara/photos

Dataset URL: https://www.kaggle.com/datasets/amanara/photos
License(s): unknown
100% 273M/274M [00:07<00:00, 12.5MB/s]
100% 274M/274M [00:07<00:00, 40.4MB/s]


In [9]:
!unzip photos.zip;

Archive:  photos.zip
replace Dataset/Testing_Data/Cat-Train (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

# Data preparation

In [48]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt

def load_training_dataset(category):
  TRAINING_DATA_FOLDER = os.path.join('Dataset', 'Training_Data', category)
  x_doc = os.path.join('Dataset', 'Training_Data', category, 'Masked_Train')
  x_imgs = _load_images_from_folder(x_doc)
  y_doc = os.path.join('Dataset', 'Training_Data', category, 'Unmasked_Train')
  y_imgs = _load_images_from_folder(y_doc)
  csv_file = pd.read_csv(os.path.join(TRAINING_DATA_FOLDER, 'masked_info.csv'))
  return x_imgs, y_imgs, csv_file

def _load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(".jpg") or filename.endswith(".jpeg"):
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
              category, id, set_type, format = _parse_filename(filename)
              images.append({'filename': filename, 'category': category, 'id': id, 'set_type': set_type, 'format': format, 'image': img})
    images_df = pd.DataFrame(images, columns=['filename', 'category', 'id', 'set_type', 'format', 'image']).set_index('filename')
    return images_df

import re

def _parse_filename(filename):
    regex_pattern = r'^(?P<Category>\w+)-(?P<Set_Type>\w+)\s\((?P<ID>\d+)\)\.(?P<Format>\w+)$'

    match = re.match(regex_pattern, filename)

    if match:
        category = match.group('Category')
        set_type = match.group('Set_Type')
        id = int(match.group('ID'))
        format = match.group('Format')
        return category, id, set_type, format
    else:
        return None, None, None, None

def show_img(img, gray = False):
  if gray:
    plt.imshow(img, 'gray')
  else:
    plt.imshow(img)
  plt.axis('off')
  plt.show()


In [27]:
categories = ['Cat', 'Dog', 'Elephant', 'Tiger']

In [49]:
x, y, csv = load_training_dataset(categories[0])

In [51]:
x.iloc[0]

category                                                  Cat
id                                                       1298
set_type                                                Train
format                                                   jpeg
image       [[[39, 71, 60], [20, 52, 41], [15, 46, 37], [1...
Name: Cat-Train (1298).jpeg, dtype: object

In [46]:
x.iloc[0]['filename']

'Cat-Train (1298).jpeg'